In [2]:
%cd code

[Errno 2] No such file or directory: 'code'
/opt/ml/code


In [5]:
from collections import Counter

from load_data import *
import pandas as pd
import pororo as Pororo

In [6]:
def add_entity_tokens(sentence, a1, a2, b1, b2):
    new_sentence = None
    ner = Pororo(task="ner", lang="ko")

    if a1 > b1:
        e1, e2 = sentence[b1:b2+1], sentence[a1:a2+1]
        n1, n2 = Counter([e[1] for e in ner(e1)]).most_common(1)[0][0], Counter([e[1] for e in ner(e2)]).most_common(1)[0][0]
        ner1, ner2 = "α" + n1 + "α", "β" + n2 + "β"
        new_sentence = sentence[:b1] + "@" + ner1 + sentence[b1:b2+1] + "@" + sentence[b2+1:a1] + "#" + ner2 + sentence[a1:a2+1] + "#" + sentence[a2+1:]
    else:
        e1, e2 = sentence[a1:a2+1], sentence[b1:b2+1]
        n1, n2 = Counter([e[1] for e in ner(e1)]).most_common(1)[0][0], Counter([e[1] for e in ner(e2)]).most_common(1)[0][0]
        ner1, ner2 = "α" + n1 + "α", "β" + n2 + "β"
        new_sentence = sentence[:a1] + "#" + ner1 + sentence[a1:a2+1] + "#" + sentence[a2+1:b1] + "@" + ner2 + sentence[b1:b2+1] + "@" + sentence[b2+1:]
    return new_sentence

In [7]:
from load_data import *
train_ori = load_data("../input/data/train/train.tsv")
train_add = load_data("../input/data/train/all_csv.tsv")
print(train_ori)
print(train_add)

                                               sentence entity_01  \
0     영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버(Land Rover)...      랜드로버   
1     선거에서 민주당은 해산 전 의석인 230석에 한참 못 미치는 57석(지역구 27석,...       민주당   
2     유럽 축구 연맹(UEFA) 집행위원회는 2014년 1월 24일에 열린 회의를 통해 ...  유럽 축구 연맹   
3     용병 공격수 챠디의 부진과 시즌 초 활약한 강수일의 침체, 시즌 중반에 영입한 세르...       강수일   
4     람캄행 왕은 1237년에서 1247년 사이 수코타이의 왕 퍼쿤 씨 인트라팃과 쓰엉 ...       람캄행   
...                                                 ...       ...   
8995  2002년 FIFA 월드컵 사우디아라비아와의 1차전에서 독일은 8-0으로 승리하였는...   사우디아라비아   
8996  일본의 2대 메이커인 토요타와 닛산은 시장 점유율을 높이기 위한 신차 개발을 계속하...       토요타   
8997  방호의의 손자 방덕룡(方德龍)은 1588년(선조 21년) 무과에 급제하고 낙안군수로...       방덕룡   
8998  LG전자는 올해 초 국내시장에 출시한 2020년형 ‘LG 그램’ 시리즈를 이달부터 ...      LG전자   
8999  전남도의회 안전건설소방위원회 차영수 의원(강진1)은 지난 14일 설 명절을 앞두고 ...       차영수   

           entity_02  label  
0                자동차     17  
1                27석      0  
2               UEFA      6  
3                공격수      2  
4          퍼쿤 씨 인트라팃 

In [13]:
print(train_add.loc[0])
print(train_add.loc[1])

sentence     카터 는 이집트 와 이스라엘 을 조정 하 여 , 캠프 데이비드 에서 안와르 사다트 ...
entity_01                                              안와르 사다트
entity_02                                                  이집트
label                                                        7
Name: 0, dtype: object
sentence     카터 는 이집트 와 이스라엘 을 조정 하 여 , 캠프 데이비드 에서 안와르 사다트 ...
entity_01                                            캠프데이비드 협정
entity_02                                                 이스라엘
label                                                       10
Name: 1, dtype: object


In [22]:
print(train_ori.groupby('label').count())

       sentence  entity_01  entity_02
label                                
0          4432       4432       4432
1            92         92         92
2           553        553        553
3            44         44         44
4           683        683        683
5           186        186        186
6           231        231        231
7           209        209        209
8           164        164        164
9           115        115        115
10          815        815        815
11           58         58         58
12           45         45         45
13           11         11         11
14           45         45         45
15          335        335        335
16           23         23         23
17          171        171        171
18            7          7          7
19            4          4          4
20          135        135        135
21           84         84         84
22           56         56         56
23           79         79         79
24          

In [23]:
print(train_add.groupby('label').count())

       sentence  entity_01  entity_02
label                                
1         11205      11205      11205
2         34615      34615      34615
4           875        875        875
7         77850      77850      77850
8          4566       4566       4566
10        13779      13779      13779
11         9650       9650       9650
12          124        124        124
15         3436       3436       3436
17        15039      15039      15039
21        10461      10461      10461
22        16601      16601      16601
33        27757      27757      27757
38         1591       1591       1591
39         2062       2062       2062


In [30]:
bodiin19 = pd.read_csv("../input/data/train/19_bornIn_city.tsv", delimiter='\t')
bodiin26 = pd.read_csv("../input/data/train/26_bornIn_country.tsv", delimiter='\t')
bodiin37 = pd.read_csv("../input/data/train/37_dienIn_city.tsv", delimiter='\t')
bodiin40 = pd.read_csv("../input/data/train/40_dienIn_country.tsv", delimiter='\t')
boddin = pd.concat([bodiin19,bodiin26,bodiin37,bodiin40])
print(boddin)

                                               sentence      entity_01  \
0                    함석헌 은 1901 년  평안북도 용천 에서 출생 했 다 .             함석헌   
1      토마스 만 은 평의원 이 며 곡물 상인 이 었 던 토마스 요한 하인리히 만 과 율...          토마스 만   
2      하인리히 뵐 ( Heinrich B ö ll , 쾰른 , 1917 년 12 월 2...         하인리히 뵐   
3      하인리히 뵐 ( Heinrich B ö ll , 쾰른 , 1917 년 12 월 2...         하인리히 뵐   
4     노무현 은 ' 1986 년  부산광역시 에서 송기인 신부 로부터 영세 를 받 아 '...            송기인   
...                                                 ...            ...   
6905  테오도어 아이케(Theodor Eicke, 1892년 10월 17일 - 1943년 ...         에른스트 룀   
6906  또한, 쿠바의 체 게바라 무덤은 체 게바라가 1967년에 볼리비아에서 전사했기 때문...          체 게바라   
6907   하렘은 왕실의 가정으로 술탄과 그의 가족들이 살았으며, 터키의 초대 대통령인 무스...  무스타파 케말 아타튀르크   
6908   드디어 1965년 5월 18일 새벽 3시 35분 시리아 보안당국은 서둘러 엘리 코...          엘리 코헨   
6909   1942년 6월 라인하르트 하이드리히가 체코에서 암살당한 뒤 독일경찰청장과 국가보...    라인하르트 하이드리히   

     entity_02  label  
0         평안북도     19  
1          뤼베크     19  
2       랑엔브로이히     19  
3           쾰른 

In [31]:
boddin_add = pd.concat([boddin,train_add])
print(boddin_add)

                                                 sentence entity_01  \
0                      함석헌 은 1901 년  평안북도 용천 에서 출생 했 다 .        함석헌   
1        토마스 만 은 평의원 이 며 곡물 상인 이 었 던 토마스 요한 하인리히 만 과 율...     토마스 만   
2        하인리히 뵐 ( Heinrich B ö ll , 쾰른 , 1917 년 12 월 2...    하인리히 뵐   
3        하인리히 뵐 ( Heinrich B ö ll , 쾰른 , 1917 년 12 월 2...    하인리히 뵐   
4       노무현 은 ' 1986 년  부산광역시 에서 송기인 신부 로부터 영세 를 받 아 '...       송기인   
...                                                   ...       ...   
229606                2. "사슴여인" (박건호 작사 / 계동균 작곡) – 03:24       박건호   
229607               9. "안녕히 계세요" (장덕 작사 / 장덕 작곡) - 02:06        장덕   
229608  2009년, 블런트는 장마크 발레가 감독을 맡고 줄리언 펠로스가 각본을 쓴 《영 빅...   줄리언 펠로스   
229609  2008년 저드 애퍼토가 제작하고 세스 로건과 에번 골드버그가 각본을 맡은 코미디 ...   에번 골드버그   
229610  영화 《라이터를 켜라》,《귀신이 산다》의 장항준, 《그해 여름》의 김은희 작가가 공...       장항준   

         entity_02  label  
0             평안북도     19  
1              뤼베크     19  
2           랑엔브로이히     19  
3               쾰른     19  
4      

In [32]:
boddin_add.to_csv('../input/data/train/all+bodi.tsv', index=False, sep='\t')

In [33]:
boddin_add_load = pd.read_csv("../input/data/train/all+bodi.tsv", delimiter='\t')
print(boddin_add_load)

                                                 sentence entity_01  \
0                      함석헌 은 1901 년  평안북도 용천 에서 출생 했 다 .        함석헌   
1        토마스 만 은 평의원 이 며 곡물 상인 이 었 던 토마스 요한 하인리히 만 과 율...     토마스 만   
2        하인리히 뵐 ( Heinrich B ö ll , 쾰른 , 1917 년 12 월 2...    하인리히 뵐   
3        하인리히 뵐 ( Heinrich B ö ll , 쾰른 , 1917 년 12 월 2...    하인리히 뵐   
4       노무현 은 ' 1986 년  부산광역시 에서 송기인 신부 로부터 영세 를 받 아 '...       송기인   
...                                                   ...       ...   
280506                2. "사슴여인" (박건호 작사 / 계동균 작곡) – 03:24       박건호   
280507               9. "안녕히 계세요" (장덕 작사 / 장덕 작곡) - 02:06        장덕   
280508  2009년, 블런트는 장마크 발레가 감독을 맡고 줄리언 펠로스가 각본을 쓴 《영 빅...   줄리언 펠로스   
280509  2008년 저드 애퍼토가 제작하고 세스 로건과 에번 골드버그가 각본을 맡은 코미디 ...   에번 골드버그   
280510  영화 《라이터를 켜라》,《귀신이 산다》의 장항준, 《그해 여름》의 김은희 작가가 공...       장항준   

         entity_02  label  
0             평안북도     19  
1              뤼베크     19  
2           랑엔브로이히     19  
3               쾰른     19  
4      